# Chapter 7
## Ensemble Learning and Random Forests
_Enemble learning_ is essentially multiple machine learning models aggregating to produce a consensus answer. If you were to do this with decision trees, this would end up being a random forests.

Here, we will discuss popular methods such as bagging, boosting, stacking, and a few others.

## Voting Classifiers
Suppose you have a group of classifers that each acchieve about 80% accuracy. One simple way to get an even better classifer is to aggregate all of the results. This majority-vote classifer is called _hard voting_. This works most effectivly when the predictors are as independent as possible.

Here's a breif example of a voting classifer using three diverse classifers.

In [3]:
from sklearn.datasets import make_moons

# TODO: Split into train and test sets
X_train, y_train = make_moons(n_samples=10000, noise=0.4)


from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

An a quick look at all of the accuracys...

In [ ]:
from sklearn.metrics import accuracy_score


for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))